<a href="https://colab.research.google.com/github/kaneelgit/stock-data/blob/main/sb2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U "tensorflow-text==2.8.*"

     |████████████████████████████████| 4.9 MB 5.2 MB/s 


In [2]:
pip install -q tf-models-official==2.7.0

     |████████████████████████████████| 1.8 MB 5.4 MB/s 
     |████████████████████████████████| 1.1 MB 42.8 MB/s 
     |████████████████████████████████| 99 kB 9.7 MB/s 
     |████████████████████████████████| 352 kB 51.0 MB/s 
     |████████████████████████████████| 237 kB 72.7 MB/s 
     |████████████████████████████████| 1.2 MB 53.4 MB/s 
     |████████████████████████████████| 596 kB 64.8 MB/s 
     |████████████████████████████████| 48.3 MB 1.4 MB/s 
     |████████████████████████████████| 43 kB 1.7 MB/s 
     |████████████████████████████████| 92 kB 11.9 MB/s 


In [3]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization

import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

In [4]:
import pandas as pd
import numpy as np

#get the dataset
#get csv file
df = pd.read_csv('https://raw.githubusercontent.com/kaneelgit/stock-data/main/dataset_news.csv')

#delete the unnamed column
del df['Unnamed: 0']
df.head(5)

,NEWS,DATE,TICKER,SP1,SP2,SP5,SP10,W1_2,W2_2,W5_2,W10_2,W1_5,W1_10,W2_5,W2_10,W5_5,W5_10,W10_5,W10_10
0,City's deal with Loews rides wave of Mid-South...,2020-03-11 15:43:00,L,39.73,37.01,33.42,31.26,0,0,0,0,0,0,0,0,0,0,0,0
1,"Engine No 1, the giant-killing hedge fund, has...",2021-06-02 20:00:00,XOM,61.18,61.45,61.05,61.05,1,1,1,1,0,0,0,0,0,0,0,0
2,Is Target Unstoppable After Another Big Earnin...,2021-05-25 10:36:00,TGT,225.30,227.37,227.27,231.34,1,1,1,1,1,2,1,2,1,1,1,1
3,Tesla champion Ark Investment outguns Wall Str...,2021-01-31 19:00:00,STT,70.25,71.60,74.77,72.92,2,1,1,1,2,2,2,2,2,1,1,1
4,Here's Why NVIDIA Stock Jumped Today,2021-05-28 19:28:00,NVDA,649.78,650.58,678.79,704.76,1,1,1,1,2,2,2,2,1,2,1,1


In [18]:
#create a list with all the ticker names 
filt_list = list(np.unique(df['TICKER'].to_list()))
filt_list.append('NVIDIA')
filt_list.append('Elon')
filt_list.append('Musk')


In [27]:
def filt_string(x):
  xsplit = x.split()
  xsplit = [word for word in xsplit if word not in filt_list]
  x = ' '.join(xsplit)
  return x

In [29]:
df['NEWS_FILT'] = df['NEWS'].apply(filt_string)

In [37]:
#create a new dataset with only the needed data
#df new
df2 = df[['NEWS_FILT', 'W5_10']]

# #change values
def change_target(x):
  if x == 2:
    x = 1
  
  return x

data = df2[(df2['W5_10'] == 0) | (df2['W5_10'] == 2)].reset_index(drop = True)
data['target'] = data['W5_10'].apply(change_target)
dataset = data[['NEWS_FILT', 'target']]


In [38]:
from sklearn.model_selection import train_test_split

#create tensorflow datasets
train_df, remaining = train_test_split(dataset, random_state = 42, train_size = 0.95, stratify = dataset.target.values)
valid_df, _ = train_test_split(remaining, random_state = 42, train_size = 0.99, stratify = remaining.target.values)

train_df.shape, valid_df.shape

((17803, 2), (928, 2))

In [40]:
AUTOTUNE = tf.data.AUTOTUNE
batch_size = 32
seed = 42

train_ds = tf.data.Dataset.from_tensor_slices((train_df['NEWS_FILT'].values, train_df['target'].values))
train_ds = train_ds.batch(batch_size)
valid_ds = tf.data.Dataset.from_tensor_slices((valid_df['NEWS_FILT'].values, valid_df['target'].values))
valid_ds = valid_ds.batch(batch_size)

In [41]:
for text_batch, label_batch in train_ds.take(1):

  for i in range(3):
    print(f'Review: {text_batch.numpy()[i]}')
    label = label_batch.numpy()[i]
    print(label)

Review: b'When Should You Buy Ross Stores, Inc. (NASDAQ:ROST)?'
0
Review: b'Zacks.com featured highlights include: ResMed, The Toro Company, Masco Corp and Landstar System'
0
Review: b'Lennar (LEN) Completes SunStreet Solar Business Sale to Sunnova'
0


In [51]:
bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8' 

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

BERT model selected           : https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3


In [52]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

In [53]:
bert_model = hub.KerasLayer(tfhub_handle_encoder)

In [54]:
bert_model = hub.KerasLayer(tfhub_handle_encoder)

In [46]:
#build classifier model
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
  return tf.keras.Model(text_input, net)

In [47]:
classifier_model = build_classifier_model()

In [56]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = tf.metrics.BinaryAccuracy()

epochs = 5
steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

classifier_model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)

In [57]:
print(f'Training model with {tfhub_handle_encoder}')
history = classifier_model.fit(x=train_ds,
                               validation_data=valid_ds,
                               epochs=epochs)

Training model with https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Epoch 1/5
557/557 [==============================] - 124s 216ms/step - loss: 0.6469 - binary_accuracy: 0.6717 - val_loss: 0.6366 - val_binary_accuracy: 0.6724
Epoch 2/5
557/557 [==============================] - 121s 218ms/step - loss: 0.6211 - binary_accuracy: 0.6758 - val_loss: 0.6039 - val_binary_accuracy: 0.6875
Epoch 3/5
557/557 [==============================] - 121s 217ms/step - loss: 0.5700 - binary_accuracy: 0.7038 - val_loss: 0.6044 - val_binary_accuracy: 0.7101
Epoch 4/5
557/557 [==============================] - 121s 217ms/step - loss: 0.5207 - binary_accuracy: 0.7343 - val_loss: 0.6158 - val_binary_accuracy: 0.7080
Epoch 5/5
557/557 [==============================] - 121s 216ms/step - loss: 0.4875 - binary_accuracy: 0.7536 - val_loss: 0.6236 - val_binary_accuracy: 0.7101


In [67]:
def print_my_examples(inputs, results):
  result_for_printing = \
    [f'input: {inputs[i]:<30} : score: {results[i][0]:.6f}'
                         for i in range(len(inputs))]
  print(*result_for_printing, sep='\n')
  print()


examples = [
    "Here's Why Stock Jumped Today", 'NVIDIA', 'TSLA'
]

# examples = news[30:40]

original_results = tf.sigmoid(classifier_model(tf.constant(examples)))

print('Results from the model in memory:')
print_my_examples(examples, original_results)

Results from the model in memory:
input: Here's Why Stock Jumped Today  : score: 0.503215
input: NVIDIA                         : score: 0.811436
input: TSLA                           : score: 0.744660



In [60]:
filt_string(examples[0])

"Here's Why Stock Jumped Today"

In [ ]:
news = dataset[dataset['target'] == 1]['NEWS'].to_list()

In [ ]:
classifier_model.save('stock_bert2', include_optimizer = False)

In [ ]:
!zip -r /content/stock_bert2.zip /content/stock_bert2

  adding: content/stock_bert2/ (stored 0%)
  adding: content/stock_bert2/assets/ (stored 0%)
  adding: content/stock_bert2/assets/vocab.txt (deflated 53%)
  adding: content/stock_bert2/saved_model.pb (deflated 92%)
  adding: content/stock_bert2/keras_metadata.pb (deflated 85%)
  adding: content/stock_bert2/variables/ (stored 0%)
  adding: content/stock_bert2/variables/variables.index (deflated 75%)
  adding: content/stock_bert2/variables/variables.data-00000-of-00001 (deflated 7%)


In [ ]:
from google.colab import files
files.download('/content/stock_bert2.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>